In [1]:
# Import packages
import yfinance as yf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as ts
from matplotlib import pyplot as plt
import statistics
import math

In [2]:
# amount of tickers used to hedge
hedge_base = int(input())

10


In [3]:
# notional to be hedged
position_size = float(input())

50000


In [4]:
# Read and print the stock tickers that make up S&P500
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]


symbol = tickers['Symbol']
symbol.head()


0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
Name: Symbol, dtype: object

In [5]:
# Get the data for this tickers from yahoo finance
data = yf.download(tickers.Symbol.to_list(),'2022-1-1','2022-12-31')
data.head()
df = pd.DataFrame(data=data)

[**********************85%****************       ]  429 of 502 completedGot error from yahoo api for ticker BRK.B, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
[*********************100%***********************]  502 of 502 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found for this date range, symbol may be delisted


In [6]:
x=df["Close"].corr()
matrix = np.triu(x)

In [7]:
close = data["Adj Close"]
log = np.log(close/close.shift())
vol = log.std()*252**.5
volatility = pd.DataFrame(vol)
volatility

,0
A,0.355268
AAL,0.567521
AAP,0.385103
AAPL,0.356963
ABBV,0.227420
...,...
YUM,0.243214
ZBH,0.307948
ZBRA,0.472296
ZION,0.384138


In [8]:
matrix = np.triu(x)


In [13]:
df2 = pd.DataFrame(data=x)
ticker = "AAPL"

In [10]:
y = df2.nlargest(hedge_base+1,[ticker])
for row in y.index:
    print(row, end= " ")
y[ticker]

TSLA AMZN EXR INVH GPN EW AVB EQR ESS AAPL LYV 

TSLA    1.000000
AMZN    0.911526
EXR     0.902323
INVH    0.896388
GPN     0.883778
EW      0.874036
AVB     0.869714
EQR     0.863718
ESS     0.857950
AAPL    0.854678
LYV     0.838743
Name: TSLA, dtype: float64

In [14]:
z = df2.nsmallest(hedge_base,[ticker])
for row in z.index:
    print(row, end= " ")


MRK VLO LLY XOM WRB MPC NOC HES CI OXY 

In [15]:
z[ticker]

MRK   -0.622813
VLO   -0.613909
LLY   -0.612889
XOM   -0.601686
WRB   -0.592190
MPC   -0.582828
NOC   -0.577988
HES   -0.543871
CI    -0.537491
OXY   -0.530202
Name: AAPL, dtype: float64

In [186]:
bigdata = pd.concat([df2[ticker],volatility], axis=1, ignore_index=True)

In [189]:
volcorr = bigdata.nsmallest(hedge_base,0)
adjvalue = volcorr.multiply(position_size)
nothedge = adjvalue.divide(hedge_base)
nothedge

,0,1
MRK,-5509.522564,1443.800387
XOM,-5389.195337,2528.959968
MPC,-5095.500213,2606.932591
PSX,-4923.761872,2824.539863
LLY,-4901.165902,1963.814020
HES,-4839.800578,3211.417862
CI,-4800.398550,1917.050055


In [179]:
df3 = z[ticker]

In [180]:
adjusted_values = df3.multiply(position_size)

In [181]:
notionalhedge = adjusted_values.divide(hedge_base)
notionalhedge

MRK   -5509.522564
XOM   -5389.195337
MPC   -5095.500213
PSX   -4923.761872
LLY   -4901.165902
HES   -4839.800578
CI    -4800.398550
Name: TSLA, dtype: float64